# 5-Day AI Agents Intensive - Setup Notebook

This notebook will help you verify your setup and prepare for the course.

## 1. Install Required Packages

Install common packages that may be needed for AI agent development.

In [2]:
# Install Google AI and related packages
%pip install -q google-generativeai google-ai-generativelanguage
%pip install -q python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 2. Set Up API Key

Store your Google AI Studio API key securely using environment variables or Kaggle Secrets.

In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file (for local development)
load_dotenv()

# For Kaggle, use Kaggle Secrets instead:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")

# For local development with .env file:
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if GOOGLE_API_KEY:
    print("✓ API Key loaded successfully")
else:
    print("✗ API Key not found. Please set up your API key.")

✓ API Key loaded successfully


## 3. Test Google Generative AI Connection

In [5]:
import google.generativeai as genai

# Configure the API key
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    
    # Test with a simple prompt
    try:
        # Use gemini-2.0-flash (current stable model)
        model = genai.GenerativeModel('gemini-2.0-flash')
        response = model.generate_content('Say hello!')
        print("✓ Connection successful!")
        print(f"Response: {response.text}")
    except Exception as e:
        print(f"✗ Error connecting to API: {e}")
        print("\nTip: Run the next cell to see available models")
else:
    print("⚠ Skipping test - API key not configured")

✓ Connection successful!
Response: Hello! How can I help you today?



## 4. List Available Models

In [6]:
if GOOGLE_API_KEY:
    try:
        print("Available Gemini models:")
        for model in genai.list_models():
            if 'generateContent' in model.supported_generation_methods:
                print(f"  - {model.name}")
    except Exception as e:
        print(f"Error listing models: {e}")
else:
    print("⚠ Skipping - API key not configured")

Available Gemini models:
  - models/gemini-2.5-pro-preview-03-25
  - models/gemini-2.5-flash-preview-05-20
  - models/gemini-2.5-flash
  - models/gemini-2.5-flash-lite-preview-06-17
  - models/gemini-2.5-pro-preview-05-06
  - models/gemini-2.5-pro-preview-06-05
  - models/gemini-2.5-pro
  - models/gemini-2.0-flash-exp
  - models/gemini-2.0-flash
  - models/gemini-2.0-flash-001
  - models/gemini-2.0-flash-exp-image-generation
  - models/gemini-2.0-flash-lite-001
  - models/gemini-2.0-flash-lite
  - models/gemini-2.0-flash-preview-image-generation
  - models/gemini-2.0-flash-lite-preview-02-05
  - models/gemini-2.0-flash-lite-preview
  - models/gemini-2.0-pro-exp
  - models/gemini-2.0-pro-exp-02-05
  - models/gemini-exp-1206
  - models/gemini-2.0-flash-thinking-exp-01-21
  - models/gemini-2.0-flash-thinking-exp
  - models/gemini-2.0-flash-thinking-exp-1219
  - models/gemini-2.5-flash-preview-tts
  - models/gemini-2.5-pro-preview-tts
  - models/learnlm-2.0-flash-experimental
  - models/ge

## 5. Setup Checklist

Verify all setup requirements:

In [7]:
print("Setup Checklist:")
print("================\n")

# Check Kaggle environment
try:
    import kaggle
    print("✓ Running on Kaggle")
except ImportError:
    print("○ Not on Kaggle (local environment)")

# Check API key
if GOOGLE_API_KEY:
    print("✓ Google API Key configured")
else:
    print("✗ Google API Key NOT configured")

# Check required packages
packages = ['google.generativeai', 'dotenv']
for package in packages:
    try:
        __import__(package.replace('-', '_'))
        print(f"✓ {package} installed")
    except ImportError:
        print(f"✗ {package} NOT installed")

print("\n📝 Remember to:")
print("   1. Phone verify your Kaggle account")
print("   2. Join Kaggle Discord: https://discord.gg/kaggle")
print("   3. Introduce yourself in #5dgai-introductions")
print("   4. Link Kaggle to Discord: https://kaggle.com/discord/confirmation")

Setup Checklist:

○ Not on Kaggle (local environment)
✓ Google API Key configured
✓ google.generativeai installed
✓ dotenv installed

📝 Remember to:
   1. Phone verify your Kaggle account
   2. Join Kaggle Discord: https://discord.gg/kaggle
   3. Introduce yourself in #5dgai-introductions
   4. Link Kaggle to Discord: https://kaggle.com/discord/confirmation


## 6. Notes Section

Use the cells below to take notes during the course:

### Common API Errors & Solutions

**Error 429: Rate Limit Exceeded**
- **Cause**: Too many requests in a short time
- **Solutions**:
  1. Use the retry function above (automatic exponential backoff)
  2. Add delays between requests: `time.sleep(1)`
  3. Reduce request frequency
  4. Consider batch processing with delays
  
**Error 404: Model Not Found**
- **Cause**: Using outdated model name (e.g., 'gemini-pro')
- **Solution**: Use current model names from cell 8 (e.g., 'gemini-2.0-flash')

**Error 400: Invalid Request**
- **Cause**: Malformed prompt or invalid parameters
- **Solution**: Check prompt format and API parameters

**Error 401/403: Authentication Failed**
- **Cause**: Invalid or missing API key
- **Solution**: Verify API key in `.env` file

**Best Practices:**
- ✅ Implement retry logic with exponential backoff
- ✅ Use global endpoint when possible
- ✅ Add appropriate delays between requests
- ✅ Monitor your quota usage
- ✅ Cache results when appropriate

In [8]:
import time
from typing import Optional

def generate_with_retry(model, prompt: str, max_retries: int = 3, initial_delay: float = 1.0) -> Optional[str]:
    """
    Generate content with automatic retry logic for handling rate limits (429 errors).
    
    Uses exponential backoff strategy:
    - 1st retry: wait 1 second
    - 2nd retry: wait 2 seconds
    - 3rd retry: wait 4 seconds
    
    Args:
        model: The generative AI model instance
        prompt: The text prompt to generate from
        max_retries: Maximum number of retry attempts (default: 3)
        initial_delay: Initial delay in seconds (default: 1.0)
    
    Returns:
        Generated text or None if all retries failed
    """
    delay = initial_delay
    
    for attempt in range(max_retries + 1):
        try:
            response = model.generate_content(prompt)
            return response.text
        
        except Exception as e:
            error_str = str(e)
            
            # Handle rate limit errors (429)
            if '429' in error_str or 'quota' in error_str.lower() or 'rate limit' in error_str.lower():
                if attempt < max_retries:
                    print(f"⚠ Rate limit hit. Retrying in {delay:.1f}s... (attempt {attempt + 1}/{max_retries})")
                    time.sleep(delay)
                    delay *= 2  # Exponential backoff
                else:
                    print(f"✗ Rate limit exceeded after {max_retries} retries")
                    print("💡 Tip: Wait a few minutes before trying again")
                    return None
            
            # Handle other errors
            else:
                print(f"✗ Error: {e}")
                return None
    
    return None


# Example usage
if GOOGLE_API_KEY:
    print("Testing retry logic with a simple prompt:\n")
    
    model = genai.GenerativeModel('gemini-2.0-flash')
    result = generate_with_retry(model, "Say 'Hello from retry logic!'")
    
    if result:
        print(f"✓ Success: {result}")
    else:
        print("Failed to generate content")
else:
    print("⚠ API key not configured - skipping retry test")

Testing retry logic with a simple prompt:

✓ Success: Hello from retry logic!



## 7. API Best Practices & Error Handling

Important tips for working with Google AI APIs during the course.

### Day 1 Notes (Nov 9, 2025)

*Add your notes here*

In [ ]:
# Day 1 code experiments

### Day 2 Notes (Nov 10, 2025)

*Add your notes here*

In [ ]:
# Day 2 code experiments

### Day 3 Notes (Nov 11, 2025)

*Add your notes here*

In [ ]:
# Day 3 code experiments

### Day 4 Notes (Nov 12, 2025)

*Add your notes here*

In [ ]:
# Day 4 code experiments

### Day 5 Notes (Nov 13, 2025)

*Add your notes here*

In [ ]:
# Day 5 code experiments